In [ ]:
from pyspark.streaming import StreamingContext
import sys

In [ ]:
outputPathPrefix="resSparkStreamingExamples"

In [ ]:
# Create a Spark Streaming Context object
ssc = StreamingContext(sc, 5)

In [ ]:
# Create a (Receiver) DStream that will connect to localhost:9999
lines = ssc.socketTextStream("localhost", 9999)

In [ ]:
# Apply the "standard" transformations to perform the word count task
# However, the "returned" RDDs are DStream RDDs
words = lines.flatMap(lambda line: line.split(" "))

In [ ]:
wordsOnes = words.map(lambda word: (word, 1))

In [ ]:
wordsCounts = wordsOnes.reduceByKey(lambda v1, v2: v1+v2)

In [ ]:
# Sort the content/the pairs by decreasing value (# of occurrences)
wordsCountsSortByKey = wordsCounts\
.transform(lambda batchRDD: batchRDD.sortBy(lambda pair: -1*pair[1]))

In [ ]:
# Print the result on the standard output
wordsCountsSortByKey.pprint()

In [ ]:
# Store the result in HDFS
wordsCountsSortByKey.saveAsTextFiles(outputPathPrefix, "")

In [ ]:
#Start the computation
ssc.start()

In [ ]:
# Run this application for 90 seconds
ssc.awaitTerminationOrTimeout(90)
ssc.stop(stopSparkContext=False)